In [1]:
!wget http://images.cocodataset.org/zips/val2014.zip
!unzip val2014.zip -d /content/images 

Streaming output truncated to the last 5000 lines.
 extracting: /content/images/val2014/COCO_val2014_000000535608.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000279145.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000243495.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000225791.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000428067.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000059383.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000558498.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000376123.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000191304.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000171062.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000014549.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000121152.jpg  
 extracting: /content/images/val2014/COCO_val2014_000000276596.jpg  
 extracting: /content/images/val2014/COCO_val2014_00

In [2]:
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [3]:
input_path = "/content/images/val2014"
output_path = "/content/VQA_preprocessed/ImagesFeaturesVAL"

In [4]:
def process_img_data(file_loc):
    raw_img_data = tf.io.read_file(file_loc)
    decoded_img = tf.image.decode_jpeg(raw_img_data, channels=3)

    # Resizing the image to 224x224 pixels
    resized_img = tf.image.resize(decoded_img, (224, 224))

    # Preprocessing the image using VGG19 preprocessing function
    preprocessed_img = tf.keras.applications.vgg19.preprocess_input(resized_img)

    return preprocessed_img, file_loc

In [5]:
def create_feature_file_path(image_path, is_val, input_path, output_path):
    full_image_path = os.path.join(input_path, image_path)
    filename = full_image_path.split('/')[-1].split('.')[0]
    feature_filename = filename + '.npy'

    if not os.path.exists(output_path):
        os.mkdir(output_path)

    subdir = 'val_data' if is_val else 'test_data'
    subdir_path = os.path.join(output_path, subdir)
    if not os.path.exists(subdir_path):
        os.mkdir(subdir_path)

    return os.path.join(subdir_path, feature_filename)

In [6]:
def generate_image_features(image_paths, is_val, feature_extractor_model, input_path, output_path):

    unique_paths = list(set(image_paths))
    print("Total images to process: ", len(unique_paths))

    image_dataset = tf.data.Dataset.from_tensor_slices(unique_paths)
    image_dataset = image_dataset.map(process_img_data, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

    for batch_images, batch_paths in tqdm(image_dataset):
        batch_features = feature_extractor_model(batch_images)
        batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

        # Save the features as NumPy files
        for features, img_path in zip(batch_features, batch_paths):
            path_str = img_path.numpy().decode("utf-8")
            feature_file_path = create_feature_file_path(path_str, is_val, input_path, output_path)
            np.save(feature_file_path, features.numpy())

In [7]:
def print_val_image_paths(val_data, input_path):
    print(f"Number of validating images: {len(val_data)}")

    for val_item in val_data:
        relative_img_path = val_item[0]
        full_img_path = os.path.join(input_path, relative_img_path)
        print(full_img_path)
        break

In [8]:
def load_val_data(data_path, input_path):
    val_data = np.load(data_path, allow_pickle=True).tolist()
    full_img_paths = []
    for item in val_data:
        relative_img_path = item[0]
        filename = relative_img_path.split('/')[-1]
        full_img_path = os.path.join(input_path, 'val2014', filename)
        full_img_paths.append(full_img_path)
    return full_img_paths

def create_feature_extractor_model():
    # Load the VGG19 model without the top classification layers
    vgg19_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    # Get the input and the output of the last layer
    input_layer = vgg19_model.input
    output_layer = vgg19_model.layers[-1].output
    # Create a new model with the input and the output of the last layer
    feature_extractor_model = tf.keras.Model(input_layer, output_layer)
    print("Image model ready")
    return feature_extractor_model

def process_val_data(data_path, input_path, output_path):
    val_img_paths = load_val_data(data_path, input_path)
    feature_extractor = create_feature_extractor_model()
    generate_image_features(val_img_paths, is_val=True, feature_extractor_model=feature_extractor, input_path=input_path, output_path=output_path)
    print("Val image features extraction done.")

data_path = '/content/drive/MyDrive/DL_Project/VQA_preprocessed/val.npy'
input_path = '/content/images'
output_path = '/content/ImagesFeaturesVAL'
process_val_data(data_path, input_path, output_path)

80134624/80134624 [==============================] - 4s 0us/step
Image model ready
Total images to process:  40504


100%|██████████| 633/633 [06:22<00:00,  1.65it/s]

Val image features extraction done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
!zip -r /content/ImagesFeaturesVAL.zip /content/drive/MyDrive/DL_Project/VQA_preprocessed/ImagesFeatures

	zip warning: name not matched: /content/drive/MyDrive/DL_Project/VQA_preprocessed/ImagesFeaturesVAL

zip error: Nothing to do! (try: zip -r /content/ImagesFeaturesVAL.zip . -i /content/drive/MyDrive/DL_Project/VQA_preprocessed/ImagesFeaturesVAL)


In [12]:
!zip -r /content/ImagesFeaturesVAL.zip /content/ImagesFeaturesVAL
!cp /content/ImagesFeaturesVAL.zip /content/drive/MyDrive/DL_Project/VQA_preprocessed


Streaming output truncated to the last 5000 lines.
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000256769.npy (deflated 84%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000288391.npy (deflated 81%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000386500.npy (deflated 81%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000553253.npy (deflated 88%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000451511.npy (deflated 84%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000021613.npy (deflated 85%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000155131.npy (deflated 86%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000185464.npy (deflated 86%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000502959.npy (deflated 85%)
  adding: content/ImagesFeaturesVAL/val_data/COCO_val2014_000000155571.npy (deflated 83%)
  adding: content/ImagesFeaturesVAL/val_data/COCO